In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path = "../../../data/util/"
df = pd.read_csv(path + "dataset_treinamento.csv")

In [ ]:
print("Quantidade de linhas antes de retirar locais com 0 acidentes", len(df))

CRIAÇÃO DE TABELA DE LOCAIS E ACIDENTES

In [ ]:
df_nv = pd.DataFrame(columns=['Local', 'Acidentes'])

for index, row in df.iterrows():
    local = row['Local de instalação']
    acidentes = row['Quantidade de Acidentes']
    if local in df_nv['Local'].values:
        df_nv.loc[(df_nv["Local"] == local), "Acidentes",] += acidentes
    else:
        df_nv = pd.concat(
            [
                df_nv,
                pd.DataFrame(
                    {
                        "Local": [local],
                        "Acidentes": [acidentes]
                    }
                ),
            ]
        )


pd.options.display.max_rows = 500
df_nv

SEPARANDO APENAS LOCAIS COM 0 ACIDENTES

In [ ]:
zero_acidentes = df_nv.loc[df_nv['Acidentes'] == 0]
zero_acidentes

TIRANDO LOCAIS QUE NAO TEM ACIDENTES DO DATASET DE ENTRADA

In [ ]:
for indx, row in df.iterrows():
    local = row['Local de instalação']
    if local in zero_acidentes['Local'].values:
        df = df.drop(index=indx)

In [ ]:
print("Quantidade de linhas depois de retirar locais com 0 acidentes", len(df))